# Members of the European Parliament (MEPs)

This code allows to extract details on Members of the European Parliament from the website of the European Parliament (https://www.europarl.europa.eu/meps/en/full-list/a).

It creates a pandas data frame including MEPs' name, political party group, date of birth, place of birth, and committee roles.

## Preparations

In [118]:
# import relevant packages
!pip install scrapethat
from scrapethat import *
import pandas as pd
from tqdm import tqdm
from google.colab import files

## Create a function to process details from one MEP profile

In [114]:
# Create a function to process details on one MEP
# Sometimes, MEP date/place of birth is missing. For those cases, insert 'missing'
def process_one_mep (link):
  t = read_cloud(link)
  data = {}

  data['Name'] = t.select_one('.sln-member-name').text.strip()
  data['Political_group'] = t.select_one('.sln-political-group-name').text.strip()

  try:
      data['Date_of_birth'] = t.select_one('.sln-birth-date').text.strip()
  except:
      data['Date_of_birth'] = 'missing'

  try:
      data['Place_of_birth'] = t.select_one('.sln-birth-place').text.strip()
  except:
      data['Place_of_birth'] = 'missing'

  length = len([x.text.strip() for x in t.select_one('.erpl_meps-status-list').select('.erpl_title-h4')])

  for i in range(0, length):
    key = t.select_one('.erpl_meps-status-list').select('.erpl_title-h4')[i].text
    value = ' # '.join([x.text.strip() for x in t.select_one('.erpl_meps-status-list').select('.erpl_meps-status')[i].select('.erpl_committee')])
    data[key] = value

  return(data)

## Retrieve the links to MEPs' profiles

In [115]:
# download all links & create one big list

all_mep_links = []

main_page_links = [f'https://www.europarl.europa.eu/meps/en/full-list/{x}' for x in list(map(chr, range(97, 123)))]

for main_page in main_page_links:
  t = read_cloud(main_page)
  mep_links = [x['href'] for x in t.select('.erpl_member-list-item-content')]
  all_mep_links.extend(mep_links)

Alternative: Use single page with all MEP profiles (https://www.europarl.europa.eu/meps/en/full-list/all)

## Create a data frame with details on all MEPs

In [116]:
# depending on scope, remove '[0:20]' or replace the indices with the IDs of interest

list_of_dfs = list(map(process_one_mep, tqdm(all_mep_links[0:20])))
df

100%|██████████| 20/20 [00:38<00:00,  1.91s/it]


,Name,Political_group,Date_of_birth,Place_of_birth,Vice-Chair,Member,Substitute,Chair
0,Mika AALTOLA,Group of the European People's Party (Christia...,02-05-1969,Jämsänkoski,Delegation to the EU-UK Parliamentary Partners...,Committee on Foreign Affairs,Committee on International Trade # Delegation ...,NaN
1,Maravillas ABADÍA JOVER,Group of the European People's Party (Christia...,08-12-1981,Murcia,NaN,Committee on Employment and Social Affairs # C...,Committee on Regional Development,NaN
2,Magdalena ADAMOWICZ,Group of the European People's Party (Christia...,10-04-1973,Słupsk,NaN,"Committee on Civil Liberties, Justice and Home...",Committee on Transport and Tourism # Committee...,NaN
3,Georgios AFTIAS,Group of the European People's Party (Christia...,missing,missing,Delegation to the EU-Serbia Stabilisation and ...,Committee on Budgets # Committee on Budgetary ...,Delegation for relations with Canada,NaN
4,Oihane AGIRREGOITIA MARTÍNEZ,Renew Europe Group,18-04-1980,Bilbao,Delegation for relations with Mercosur,Committee on Transport and Tourism # Delegatio...,"Committee on Industry, Research and Energy # C...",NaN
5,Peter AGIUS,Group of the European People's Party (Christia...,09-04-1979,St. Julians,NaN,Committee on the Internal Market and Consumer ...,Committee on Agriculture and Rural Development,NaN
6,Alex AGIUS SALIBA,Group of the Progressive Alliance of Socialist...,31-01-1988,Pieta,NaN,Committee on the Internal Market and Consumer ...,Committee on Employment and Social Affairs # C...,NaN
7,Galato ALEXANDRAKI,European Conservatives and Reformists Group,missing,missing,NaN,Committee on Women’s Rights and Gender Equalit...,"Committee on the Environment, Public Health an...",NaN
8,Grégory ALLIONE,Renew Europe Group,25-07-1971,Toulon,NaN,Committee on Employment and Social Affairs # C...,Delegation for relations with the countries of...,NaN
9,Abir AL-SAHLANI,Renew Europe Group,18-05-1976,Basra,Committee on Development # Delegation for rela...,"Committee on Civil Liberties, Justice and Home...",Delegation for relations with South Africa,NaN


In [120]:
df.to_csv('mep.csv', encoding = 'utf-8-sig', index=False)
files.download('mep.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>